In [2]:
import os
from os.path import join, isfile
import numpy as np
import h5py
from glob import glob
# from torch.utils.serialization import load_lua  
from PIL import Image 
import yaml
import torchfile
import io
import pdb

In [4]:
torchfile.load('/Users/rajibdas/Workspace/UB/CV/data/flowers_icml/class_00001/image_06734.t7')[b'word']

array([[ 292, 5383,  292,  292, 5383, 5383,  851, 5383, 5383, 5383],
       [  36, 2805, 1870, 4852, 2805, 2805, 2805, 2805, 2805, 2805],
       [4630,  732, 5253,   36, 2373, 2373, 1285, 2373, 2373, 2373],
       [ 292, 4608, 4630,  611, 5230,   36, 4740,  408, 5230, 4852],
       [2805, 2707,  292, 2466, 4608, 1121, 2707, 4608, 4720,   36],
       [4476,  381, 5363, 4630,   36, 4476, 2441,   36, 4608, 1034],
       [4608,  890, 4852,  381, 2707, 4608,   36, 1285,   36, 3820],
       [ 890, 2085,   36, 1285,  851, 2707, 1121,  851, 1285, 1034],
       [2085, 1285, 1938,  381,  381, 2373,  732,  999, 2589,  851],
       [2707,   36, 1035, 4450, 1608,  851, 1907, 5231, 1972,  381],
       [ 611, 1121,  292, 2707,  890,  999, 4852,    1, 2707, 3976],
       [ 851, 4476, 4164,  108,  292, 3976,    1,    1, 4852,    1],
       [ 999,  793, 3976,    1, 5231,    1,    1,    1, 5026,    1],
       [5231, 1444, 2707,    1,    1,    1,    1,    1,    1,    1],
       [   1,    1,  108,    1,   

In [2]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

images_path = config['birds_images_path']
embedding_path = config['birds_embedding_path']
text_path = config['birds_text_path']
datasetDir = config['birds_dataset_path']

val_classes = open(config['val_split_path']).read().splitlines()
train_classes = open(config['train_split_path']).read().splitlines()
test_classes = open(config['test_split_path']).read().splitlines()

f = h5py.File(datasetDir, 'w')
train = f.create_group('train')
valid = f.create_group('valid')
test = f.create_group('test')


In [6]:
for _class in sorted(os.listdir(embedding_path)):
    _class = _class.rstrip(".t7")
    split = ''
    if _class in train_classes:
        split = train
    elif _class in val_classes:
        split = valid
    elif _class in test_classes:
        split = test
        
    data_path = os.path.join(embedding_path, _class)
    txt_path = os.path.join(text_path, _class)
    img_path = os.path.join(images_path, _class)
    
    data = torchfile.load(data_path)
    img = torchfile.load(img_path)
    

    for i, txt_file in enumerate(sorted(glob(txt_path + "/*.txt"))):
		# img_path = example_data['img']
		# embeddings = example_data['txt'].numpy()
		# example_name = img_path.split('/')[-1][:-4]
        
        f = open(txt_file, "r")
        txt = f.readlines()
        f.close()
        
		# img_path = os.path.join(images_path, img_path)
		# img = open(img_path, 'rb').read()

		# txt_choice = np.random.choice(range(10), 5)

		# embeddings = embeddings[txt_choice]
		# txt = np.array(txt)
		# txt = txt[txt_choice]
		# dt = h5py.special_dtype(vlen=str)

		# for c, e in enumerate(embeddings):
		# 	ex = split.create_group(example_name + '_' + str(c))
		# 	ex.create_dataset('name', data=example_name)
		# 	ex.create_dataset('img', data=np.void(img))
		# 	ex.create_dataset('embeddings', data=e)
		# 	ex.create_dataset('class', data=_class)
		# 	ex.create_dataset('txt', data=txt[c].astype(object), dtype=dt)

		# print(example_name)





/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/001.Black_footed_Albatross
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/002.Laysan_Albatross
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/003.Sooty_Albatross
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/004.Groove_billed_Ani
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/010.Red_winged_Blackbird
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/011.Rusty_Blackbird
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/012.Yellow_headed_Blackbird
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/013.Bobolink
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/014.Indigo_Bunting
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/015.Lazuli_Bunting
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/016.Painted_Bunting
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10/017.Cardinal
/Users/rajibdas/Workspace/UB/CV/data/cvpr2016_cub/w2v_c10

Black_Footed_Albatross_0001_796111


In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

class generator(nn.Module):
	def __init__(self):
		super(generator, self).__init__()
		self.image_size = 64
		self.num_channels = 3
		self.noise_dim = 100
		self.ngf = 64

		# based on: https://github.com/pytorch/examples/blob/master/dcgan/main.py
		self.netG = nn.Sequential(
			nn.ConvTranspose2d(self.noise_dim, self.ngf * 8, 4, 1, 0, bias=False),
			nn.BatchNorm2d(self.ngf * 8),
			nn.ReLU(True),
			# state size. (ngf*8) x 4 x 4
			# nn.ConvTranspose2d(self.ngf * 8, self.ngf * 4, 4, 2, 1, bias=False),
			# nn.BatchNorm2d(self.ngf * 4),
			# nn.ReLU(True),
			# # state size. (ngf*4) x 8 x 8
			# nn.ConvTranspose2d(self.ngf * 4, self.ngf * 2, 4, 2, 1, bias=False),
			# nn.BatchNorm2d(self.ngf * 2),
			# nn.ReLU(True),
			# # state size. (ngf*2) x 16 x 16
			# nn.ConvTranspose2d(self.ngf * 2,self.ngf, 4, 2, 1, bias=False),
			# nn.BatchNorm2d(self.ngf),
			# nn.ReLU(True),
			# # state size. (ngf) x 32 x 32
			# nn.ConvTranspose2d(self.ngf, self.num_channels, 4, 2, 1, bias=False),
			# nn.Tanh()
			 # state size. (num_channels) x 64 x 64
			)

	def forward(self, z):
		return self.netG(z)


class discriminator(nn.Module):
	def __init__(self):
		super(discriminator, self).__init__()
		self.image_size = 64
		self.num_channels = 3
		self.ndf = 64

		self.B_dim = 128
		self.C_dim = 16

		self.netD_1 = nn.Sequential(
			# input is (nc) x 64 x 64
			nn.Conv2d(self.num_channels, self.ndf, 4, 2, 1, bias=False),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf) x 32 x 32
			nn.Conv2d(self.ndf, self.ndf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 2),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*2) x 16 x 16
			nn.Conv2d(self.ndf * 2, self.ndf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 4),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*4) x 8 x 8
			nn.Conv2d(self.ndf * 4, self.ndf * 8, 4, 2, 1, bias=False),
			nn.BatchNorm2d(self.ndf * 8),
			nn.LeakyReLU(0.2, inplace=True)
		)

		self.netD_2 = nn.Sequential(
			# state size. (ndf*8) x 4 x 4
			nn.Conv2d(self.ndf * 8, 1, 4, 1, 0, bias=False),
			nn.Sigmoid()
			)

	def forward(self, inp):
		x_intermediate = self.netD_1(inp)
		output =  self.netD_2(x_intermediate)
		return output.view(-1, 1).squeeze(1), x_intermediate


In [4]:
gen = generator()